In [1]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F, types as T
from pyspark.rdd import RDD

import os

home_dir = os.path.expanduser("~")
print(home_dir)

/home/ppaton


In [2]:
import pyspark
print(pyspark.__version__)

3.5.5


In [3]:
spark: SparkSession = SparkSession.builder \
    .appName("GPU RAPIDS SQL Example") \
    .master("local[*]") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.rapids.sql.explain", "ALL") \
    .config("spark.executor.resource.gpu.amount", "1") \
    .config("spark.driver.extraClassPath", f"{home_dir}/rapids_jars/rapids-4-spark_2.12-25.04.0.jar:{home_dir}/rapids_jars/cudf-25.04.0-cuda12.jar") \
    .config("spark.executor.extraClassPath", f"{home_dir}/rapids_jars/rapids-4-spark_2.12-25.04.0.jar:{home_dir}/rapids_jars/cudf-25.04.0-cuda12.jar") \
    .getOrCreate()

sc = spark.sparkContext
print(f"Monitor cluster at: {sc.uiWebUrl}")

25/06/01 18:34:19 WARN Utils: Your hostname, ppaton-desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.174 instead (on interface wlp3s0)
25/06/01 18:34:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/01 18:34:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/01 18:34:19 WARN ResourceProfile: The executor resource config for resource: gpu was specified but no corresponding task resource request was specified.
25/06/01 18:34:20 WARN RapidsPluginUtils: RAPIDS Accelerator 25.04.0 using cudf 25.04.0, private revision 74d87a45eb421fc84a84c6997b48c734ebe6556e
25/06/01 18:34:20 WARN RapidsPluginUtils: Multiple cudf jars found in the classpath:
revison: 6bc420631d12b0e7d87c5ede7710ac5b7154fd8d
	jar URL: jar:file:/home/ppaton/rapids_jars/r

Monitor cluster at: http://ppaton-desktop.Home:4040


### Generate and process resilient distributed dataset (RDD) from an array on the driver

In [4]:
data = [1, 2, 3, 4, 5]
rdd: RDD[int] = sc.parallelize(data)
result = rdd.map(lambda x: x * 2).collect()
print(result)  # Output: [2, 4, 6, 8, 10]

[2, 4, 6, 8, 10]


### Generate and process a DataFrame from a .csv file on the driver 

In [5]:
higgs_data: DataFrame = spark.read.csv(
    "../data/HIGGS.csv",
    header=False,
    inferSchema=True
)
higgs_data.printSchema()

25/06/01 18:34:52 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
  *Exec <FilterExec> will run on GPU
    *Expression <GreaterThan> (length(trim(value#0, None)) > 0) will run on GPU
      *Expression <Length> length(trim(value#0, None)) will run on GPU
        *Expression <StringTrim> trim(value#0, None) will run on GPU
    !Exec <FileSourceScanExec> cannot run on GPU because unsupported file format: org.apache.spark.sql.execution.datasources.text.TextFileFormat

25/06/01 18:34:54 WARN GpuOverrides:                                            
! <DeserializeToObjectExec> cannot run on GPU because not a

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tru

In [6]:
(
    higgs_data
    .agg(
        F.mean(F.col("_c0")).alias("mean_c0"),
        F.stddev(F.col("_c0")).alias("stddev_c0"),
    )
    .toPandas()
)

25/06/01 18:35:44 WARN GpuOverrides: 
*Exec <HashAggregateExec> will run on GPU
  *Expression <AggregateExpression> avg(_c0#17) will run on GPU
    *Expression <Average> avg(_c0#17) will run on GPU
  *Expression <AggregateExpression> stddev(_c0#17) will run on GPU
    *Expression <StddevSamp> stddev(_c0#17) will run on GPU
  *Expression <Alias> avg(_c0#17)#75 AS mean_c0#76 will run on GPU
  *Expression <Alias> stddev(_c0#17)#115 AS stddev_c0#77 will run on GPU
  *Exec <ShuffleExchangeExec> will run on GPU
    *Partitioning <SinglePartition$> will run on GPU
    *Exec <HashAggregateExec> will run on GPU
      *Expression <AggregateExpression> partial_avg(_c0#17) will run on GPU
        *Expression <Average> avg(_c0#17) will run on GPU
      *Expression <AggregateExpression> partial_stddev(_c0#17) will run on GPU
        *Expression <StddevSamp> stddev(_c0#17) will run on GPU
      *Exec <FileSourceScanExec> will run on GPU

25/06/01 18:35:44 WARN GpuOverrides: 
*Exec <HashAggregateExec>

,mean_c0,stddev_c0
0,0.52992,0.499104


### Free cluster resources

In [7]:
spark.stop()